This script is meant to replicate the block that reading brain activity from neurons and processing it to provide the BMI with spike binned information.

### Imports

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import paho.mqtt.client as paho
import numpy as np
from time import sleep
from datetime import datetime
import json
import pickle as pkl

from utils.functions import *


### Parameters

In [3]:
clientID = 'brain_decoder'
broker = 'localhost'
port = 1883
topic = 'brain_activity'

### Script

In [4]:
# Initialize the publisher
client = paho.Client(clientID, False)

In [13]:
# Load the testing samples
test_trials = pkl.load(open('../data/online/test_dict_10.pkl', 'rb'))
print(test_trials.keys())

dict_keys(['boxes', 'cubes', 'balls', 'hCylinder', 'rings', 'vCylinder'])


In [16]:
# Start the script, waiting for user input (should be the test to perform)
while True:
    action = input(f'Which trial to simulate?\n{str(test_trial.keys())[11:-2]} or exit:\t')
    action = action.strip().lower()
    if action in ['exit', 'e']:
        break
        
    if action not in test_trial.keys():
        print('Incorrect input')
        continue
        
    # select the desired trial
    trial = test_trials[action]
    (channels, nbins) = trial.shape
    
    # start the client 
    client.connect(broker, port)
    client.loop_start()
    client.publish(topic, json.dumps({"status": "starting_simulation", "shape": channels}), 2)
    
    try:
        for idx in progressbar(range(nbins), "Simulating: ", 60):
            time_bin = np.squeeze(trial[:, idx])
            # Every 10ms prepare a new json with status and column of the test until finishing the trial 
            msg = json.dumps({
                "name": "Binned brain activity",
                "description": "Recording from all the channels with datetime and simulation index",
                "value": {
                    "measurements": time_bin.tolist(),
                    "datetime": datetime.now().strftime('%Y-%m-%dT%H:%M:%S'),
                    "index": idx
                },
                "status": "running_simulation"
            })
            client.publish(topic, msg, 2)
            sleep(1)
            #sleep(0.01)
            
        client.publish(topic, json.dumps({"status": "ending_simulation"}), 2)
        client.loop_stop()
        client.disconnect()
    
    except KeyboardInterrupt:
        print('Simulation interrupted')
        client.loop_stop()
        client.disconnect()
        sys.exit(0)
    
    print('\n')

Which trial to simulate?
'boxes', 'cubes', 'balls', 'hCylinder', 'rings', 'vCylinder' or exit:	 boxes


Simulation interrupted...................................................] 10/139


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-16-65e31a1ccf03>", line 36, in <module>
    sleep(1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\miniconda3\envs\thesis\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-65e31a1ccf03>", line 47, in <module>
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\miniconda3\envs\thesis\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "D:\miniconda3\envs\thesis\lib\site-packages\IPython\core\ultratb.py", line 248, in wrapped
    return f(*args, **kwargs)
  File "D:\miniconda3\envs\thesis\lib\site-packages\IPython\core\ultratb.py", line 281, in _fi

TypeError: object of type 'NoneType' has no len()